In [1]:
import os
import urllib.parse
from datetime import datetime
from pathlib import Path

import pandas as pd
from sqlalchemy import create_engine, text
import pyodbc, sys, platform
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
# requerimientos
print("Python:", sys.version)
print("Arquitectura:", platform.architecture())
print("Drivers ODBC disponibles:", pyodbc.drivers())


Python: 3.12.10 (tags/v3.12.10:0cc8128, Apr  8 2025, 12:21:36) [MSC v.1943 64 bit (AMD64)]
Arquitectura: ('64bit', 'WindowsPE')
Drivers ODBC disponibles: ['SQL Server', 'Microsoft Access Driver (*.mdb, *.accdb)', 'Microsoft Excel Driver (*.xls, *.xlsx, *.xlsm, *.xlsb)', 'Microsoft Access Text Driver (*.txt, *.csv)', 'ODBC Driver 13 for SQL Server', 'ODBC Driver 18 for SQL Server']


# Conexion API

In [3]:
# === 1) Parámetros  ===
SERVER   = os.getenv("SINCO_SERVER")
DATABASE = os.getenv("SINCO_DB")
USER     = os.getenv("SINCO_USER")
PASSWORD = os.getenv("SINCO_PW")
DRIVER   = "ODBC Driver 18 for SQL Server"

vars = [SERVER, DATABASE, USER, PASSWORD]

# for var in vars:
#     print(var)

In [4]:
# === 2) Crear SQLAlchemy engine (evita el warning de pandas) ===
odbc_params = (
    f"DRIVER={{{DRIVER}}};"
    f"SERVER={SERVER};"
    f"DATABASE={DATABASE};"
    f"UID={USER};PWD={PASSWORD};"
    "Encrypt=yes;TrustServerCertificate=yes;"
)

conn_str = "mssql+pyodbc:///?odbc_connect=" + urllib.parse.quote_plus(odbc_params)
engine = create_engine(conn_str, fast_executemany=True)

In [5]:
# === 3) Directorio de exportación ===
out_dir = Path("export") / datetime.now().strftime("%Y%m%d")
out_dir.mkdir(parents=True, exist_ok=True)


# Utilidades

In [6]:
# === 4) Utilidades ===
def listar_tablas(esquemas=None):
    """
    Retorna DataFrame con tablas BASE (no vistas). Filtra por lista de esquemas opcional.
    """
    q = """
    SELECT TABLE_SCHEMA, TABLE_NAME
    FROM INFORMATION_SCHEMA.TABLES
    WHERE TABLE_TYPE = 'BASE TABLE'
    ORDER BY TABLE_SCHEMA, TABLE_NAME;
    """
    df = pd.read_sql(text(q), engine)
    if esquemas:
        df = df[df["TABLE_SCHEMA"].isin(esquemas)]
    return df.reset_index(drop=True)

def exportar_tabla(schema, table, chunksize=200_000, to_parquet=False):
    """
    Exporta una tabla completa en chunks a CSV (y opcional Parquet).
    Archivo: export/YYYYMMDD/schema.table.csv
    """
    dest_csv = out_dir / f"{schema}.{table}.csv"
    dest_parq = out_dir / f"{schema}.{table}.parquet"

    sql = text(f'SELECT * FROM "{schema}"."{table}"')  # comillas dobles por seguridad
    first = True
    rows = 0

    with engine.connect() as conn:
        for chunk in pd.read_sql(sql, conn, chunksize=chunksize):
            mode = "w" if first else "a"
            header = first
            chunk.to_csv(dest_csv, index=False, mode=mode, header=header)
            rows += len(chunk)
            first = False

    if to_parquet:
        # Si quieres Parquet, lo armamos leyendo de nuevo el CSV (o podrías acumular en memoria si cabe)
        df = pd.read_csv(dest_csv)
        df.to_parquet(dest_parq, index=False)

    return dest_csv, rows

def exportar_todas(esquemas=None, to_parquet=False):
    tablas = listar_tablas(esquemas)
    print(f"Encontradas {len(tablas)} tablas.")
    resumen = []
    for _, r in tablas.iterrows():
        s, t = r.TABLE_SCHEMA, r.TABLE_NAME
        print(f"→ Exportando {s}.{t} ...", end="", flush=True)
        try:
            path, n = exportar_tabla(s, t, to_parquet=to_parquet)
            print(f" OK ({n:,} filas) → {path.name}")
            resumen.append({"schema": s, "table": t, "rows": n, "file": path.name})
        except Exception as e:
            print(f" ERROR: {e}")
            resumen.append({"schema": s, "table": t, "rows": None, "file": None, "error": str(e)})
    pd.DataFrame(resumen).to_csv(out_dir / "_resumen_export.csv", index=False)
    print(f"\nResumen guardado en: {out_dir / '_resumen_export.csv'}")

## diccionario de datos funciones

In [7]:
from sqlalchemy import text
import pandas as pd

def columnas_y_llaves(esquemas=None):
    """
    Devuelve un DataFrame por columna con:
      esquema, nombre_tabla, nombre_columna, tipo_llave (PK/UK/FK/NINGUNA), relacion_aTabla (solo FK)
    Si 'esquemas' es una lista, filtra por esos schemas.
    """
    params = {}
    filtro_cols = ""
    filtro_pkuk = ""
    filtro_fk   = ""

    if esquemas:
        ph = ", ".join([f":s{i}" for i in range(len(esquemas))])
        params.update({f"s{i}": s for i, s in enumerate(esquemas)})
        filtro_cols = f"WHERE c.TABLE_SCHEMA IN ({ph})"
        filtro_pkuk = f" AND tc.TABLE_SCHEMA IN ({ph})"
        filtro_fk   = f" AND tc.TABLE_SCHEMA IN ({ph})"

    sql = text(f"""
    WITH cols AS (
        SELECT c.TABLE_SCHEMA, c.TABLE_NAME, c.COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS c
        {filtro_cols}
    ),
    pkuk AS (
        SELECT
            tc.TABLE_SCHEMA, tc.TABLE_NAME, kcu.COLUMN_NAME,
            CASE WHEN tc.CONSTRAINT_TYPE='PRIMARY KEY' THEN 'PK' ELSE 'UK' END AS tipo_llave
        FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
        JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
          ON kcu.CONSTRAINT_NAME   = tc.CONSTRAINT_NAME
         AND kcu.CONSTRAINT_SCHEMA = tc.CONSTRAINT_SCHEMA
         AND kcu.TABLE_NAME        = tc.TABLE_NAME
        WHERE tc.CONSTRAINT_TYPE IN ('PRIMARY KEY','UNIQUE')
        {filtro_pkuk}
    ),
    fks AS (
        SELECT
            tc.TABLE_SCHEMA, tc.TABLE_NAME, kcu.COLUMN_NAME,
            'FK' AS tipo_llave,
            rtab.TABLE_SCHEMA + '.' + rtab.TABLE_NAME AS relacion_aTabla
        FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc              -- FK de la tabla "hija"
        JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
          ON kcu.CONSTRAINT_NAME   = tc.CONSTRAINT_NAME
         AND kcu.CONSTRAINT_SCHEMA = tc.CONSTRAINT_SCHEMA
         AND kcu.TABLE_NAME        = tc.TABLE_NAME
        JOIN INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS rc
          ON rc.CONSTRAINT_NAME    = tc.CONSTRAINT_NAME
         AND rc.CONSTRAINT_SCHEMA  = tc.CONSTRAINT_SCHEMA
        JOIN INFORMATION_SCHEMA.TABLE_CONSTRAINTS rtab            -- PK/UK de la tabla "padre"
          ON rtab.CONSTRAINT_NAME   = rc.UNIQUE_CONSTRAINT_NAME
         AND rtab.CONSTRAINT_SCHEMA = rc.UNIQUE_CONSTRAINT_SCHEMA
        WHERE tc.CONSTRAINT_TYPE = 'FOREIGN KEY'
        {filtro_fk}
    )
    SELECT
        c.TABLE_SCHEMA  AS esquema,
        c.TABLE_NAME    AS nombre_tabla,
        c.COLUMN_NAME   AS nombre_columna,
        COALESCE(pkuk.tipo_llave, fks.tipo_llave, 'NINGUNA') AS tipo_llave,
        fks.relacion_aTabla
    FROM cols c
    LEFT JOIN pkuk ON pkuk.TABLE_SCHEMA = c.TABLE_SCHEMA
                  AND pkuk.TABLE_NAME   = c.TABLE_NAME
                  AND pkuk.COLUMN_NAME  = c.COLUMN_NAME
    LEFT JOIN fks  ON fks.TABLE_SCHEMA  = c.TABLE_SCHEMA
                  AND fks.TABLE_NAME    = c.TABLE_NAME
                  AND fks.COLUMN_NAME   = c.COLUMN_NAME
    ORDER BY esquema, nombre_tabla, nombre_columna;
    """)

    with engine.connect() as conn:
        return pd.read_sql(sql, conn, params=params)


In [8]:
from sqlalchemy import text
import pandas as pd

def columnas_pk_fk(esquemas=None):
    """
    Devuelve por COLUMNA:
      esquema, nombre_tabla, nombre_columna, tipo_llave (PK/FK/None), relacion_aTabla (solo para FK)

    - Usa INFORMATION_SCHEMA
    - Filtra por lista de esquemas si se pasa `esquemas=[...]`
    """
    params = {}
    filtro_cols = ""
    filtro_pk   = ""
    filtro_fk   = ""

    if esquemas:
        ph = ", ".join([f":s{i}" for i in range(len(esquemas))])
        params.update({f"s{i}": s for i, s in enumerate(esquemas)})
        filtro_cols = f"WHERE c.TABLE_SCHEMA IN ({ph})"
        filtro_pk   = f" AND tc.TABLE_SCHEMA IN ({ph})"
        filtro_fk   = f" AND tc.TABLE_SCHEMA IN ({ph})"

    sql = text(f"""
    WITH cols AS (
        SELECT c.TABLE_SCHEMA, c.TABLE_NAME, c.COLUMN_NAME
        FROM INFORMATION_SCHEMA.COLUMNS c
        {filtro_cols}
    ),
    pk AS (
        SELECT
            tc.TABLE_SCHEMA, tc.TABLE_NAME, kcu.COLUMN_NAME
        FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc
        JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
          ON kcu.CONSTRAINT_NAME   = tc.CONSTRAINT_NAME
         AND kcu.CONSTRAINT_SCHEMA = tc.CONSTRAINT_SCHEMA
         AND kcu.TABLE_NAME        = tc.TABLE_NAME
        WHERE tc.CONSTRAINT_TYPE = 'PRIMARY KEY'
        {filtro_pk}
    ),
    fk AS (
        SELECT
            tc.TABLE_SCHEMA, tc.TABLE_NAME, kcu.COLUMN_NAME,
            rtab.TABLE_SCHEMA + '.' + rtab.TABLE_NAME AS relacion_aTabla
        FROM INFORMATION_SCHEMA.TABLE_CONSTRAINTS tc          -- constraint hija
        JOIN INFORMATION_SCHEMA.KEY_COLUMN_USAGE kcu
          ON kcu.CONSTRAINT_NAME   = tc.CONSTRAINT_NAME
         AND kcu.CONSTRAINT_SCHEMA = tc.CONSTRAINT_SCHEMA
         AND kcu.TABLE_NAME        = tc.TABLE_NAME
        JOIN INFORMATION_SCHEMA.REFERENTIAL_CONSTRAINTS rc
          ON rc.CONSTRAINT_NAME    = tc.CONSTRAINT_NAME
         AND rc.CONSTRAINT_SCHEMA  = tc.CONSTRAINT_SCHEMA
        JOIN INFORMATION_SCHEMA.TABLE_CONSTRAINTS rtab        -- constraint padre (PK/UK)
          ON rtab.CONSTRAINT_NAME   = rc.UNIQUE_CONSTRAINT_NAME
         AND rtab.CONSTRAINT_SCHEMA = rc.UNIQUE_CONSTRAINT_SCHEMA
        WHERE tc.CONSTRAINT_TYPE = 'FOREIGN KEY'
        {filtro_fk}
    )
    SELECT
        c.TABLE_SCHEMA  AS esquema,
        c.TABLE_NAME    AS nombre_tabla,
        c.COLUMN_NAME   AS nombre_columna,
        CASE
          WHEN pk.COLUMN_NAME IS NOT NULL THEN 'PK'
          WHEN fk.COLUMN_NAME IS NOT NULL THEN 'FK'
          ELSE NULL
        END AS tipo_llave,
        CASE
          WHEN fk.COLUMN_NAME IS NOT NULL THEN fk.relacion_aTabla
          ELSE NULL
        END AS relacion_aTabla
    FROM cols c
    LEFT JOIN pk ON pk.TABLE_SCHEMA = c.TABLE_SCHEMA
                AND pk.TABLE_NAME   = c.TABLE_NAME
                AND pk.COLUMN_NAME  = c.COLUMN_NAME
    LEFT JOIN fk ON fk.TABLE_SCHEMA = c.TABLE_SCHEMA
                AND fk.TABLE_NAME   = c.TABLE_NAME
                AND fk.COLUMN_NAME  = c.COLUMN_NAME
    ORDER BY esquema, nombre_tabla, nombre_columna;
    """)

    with engine.connect() as conn:
        return pd.read_sql(sql, conn, params=params)


# Revisión 

In [9]:
tablas = listar_tablas()

In [17]:
tablas

,TABLE_SCHEMA,TABLE_NAME
0,ADI_DTM,Acuerdos_Pago
1,ADI_DTM,Acuerdos_Pago_DesistimientosVenta
2,ADI_DTM,Alternos
3,ADI_DTM,Auditoria_LogVentas
4,ADI_DTM,AuditoriaRegistrosTablas
5,ADI_DTM,CamposAdicionalesCompradores
6,ADI_DTM,CamposAdicionalesVisitantes
7,ADI_DTM,CamposAdicionalesVisitas
8,ADI_DTM,Comisiones
9,ADI_DTM,Comprador


In [13]:
tablas.shape

(326, 2)

In [11]:
tablas['TABLE_SCHEMA'].unique()

array(['ADI_DTM', 'ADP_DTM', 'ADP_DTM_CONF', 'ADP_DTM_DIM',
       'ADP_DTM_FACT', 'DTM_CONF', 'DWH', 'DWHPruebas',
       'DWHSincoARPRODOMReal_Nueva', 'FIN_DTM'], dtype=object)

In [11]:
tablas[tablas['TABLE_SCHEMA'].isin( ['ADP_DTM_DIM',
       'ADP_DTM_FACT'])].shape

(50, 2)

In [22]:
tablasUsar = tablas[tablas['TABLE_SCHEMA'].isin( ['ADP_DTM_DIM','ADP_DTM_FACT'])]

In [28]:
tablasUsar.columns

Index(['TABLE_SCHEMA', 'TABLE_NAME'], dtype='object')

## Exportacion tablas

In [ ]:
# tabla = 0
# for index, row in tablasUsar.iterrows():
#     tabla+=1
#     print(f"{tabla}. {row["TABLE_SCHEMA"]}, {row["TABLE_NAME"]}")
#     exportar_tabla(row["TABLE_SCHEMA"], row["TABLE_NAME"])
#     print(f"La tabla {row["TABLE_SCHEMA"]}.{row["TABLE_NAME"]} fue exportada \n")

1. ADP_DTM_DIM, Actividades
La tabla ADP_DTM_DIM.Actividades fue exprtada 

2. ADP_DTM_DIM, Bodega
La tabla ADP_DTM_DIM.Bodega fue exprtada 

3. ADP_DTM_DIM, CapituloPresupuesto
La tabla ADP_DTM_DIM.CapituloPresupuesto fue exprtada 

4. ADP_DTM_DIM, ControlClaseOrigen
La tabla ADP_DTM_DIM.ControlClaseOrigen fue exprtada 

5. ADP_DTM_DIM, Empresa
La tabla ADP_DTM_DIM.Empresa fue exprtada 

6. ADP_DTM_DIM, EspecicficacionDePedidos
La tabla ADP_DTM_DIM.EspecicficacionDePedidos fue exprtada 

7. ADP_DTM_DIM, EspecificacionDeActas
La tabla ADP_DTM_DIM.EspecificacionDeActas fue exprtada 

8. ADP_DTM_DIM, EspecificacionDeContratos
La tabla ADP_DTM_DIM.EspecificacionDeContratos fue exprtada 

9. ADP_DTM_DIM, EspecificacionDeEntradasAlmacen
La tabla ADP_DTM_DIM.EspecificacionDeEntradasAlmacen fue exprtada 

10. ADP_DTM_DIM, EspecificacionEjecucionCliente
La tabla ADP_DTM_DIM.EspecificacionEjecucionCliente fue exprtada 

11. ADP_DTM_DIM, EstadoEnvioDocumento
La tabla ADP_DTM_DIM.EstadoEnvioDocum

In [17]:
schema = "ADI_DTM"
table =	"Proyectos"
exportar_tabla(schema, table)

(WindowsPath('export/20251001/ADI_DTM.Proyectos.csv'), 19)

# Prueba

In [19]:
proyectos = pd.read_csv(r"C:\Users\Administrador\Desktop\reto\export\20251001\ADI_DTM.Proyectos.csv")

In [20]:
proyectos.shape

(19, 47)

In [24]:
proyectos.head(5)

,PryEmpresa,PryNombreEmpresa,PryCodigoProyecto,PryNombreProyecto,PryCodigoCentroCosto,PryNombreCentroCosto,PryCodigoClaseProyecto,PryNombreClaseProyecto,PryCodigoEstadoProyecto,PryNombreEstadoProyecto,PryCodMacroProy,PryDescMacroProy,PryNaturaleza,PryNombreCiudad,PryEtapa,PryEstrato,PryInterior,PryFechaEntrega,PryFechaFinaliza,PryEntidadCredito,PryNITEntidadCredito,PryEntidadFiduciaria,PryNITEntidadFiduciaria,PryVIS,PryDireccion,PryFechaInicialVenta,PryFechaCorteCuotas,PryFechaPermiso,PryFechaAprobacionCredito,PryFechaPermisoVenta,PryFechaPromesaCompraventa,PryFechaEscrituracion,PryFechaRevision,PryFechaLicencia,PryFechaPropiedadHorizontal,PryFechaPuntoEquilibrioIni,PryFechaPuntoEquilibrioFin,PryFechaPuntoEquilibrioReal,PryFechaInicialObra,PryFechaVencimientoEntrega,PryNombreApoderado,PryNumeroCedulaApoderado,PryLugarExpedicionApoderado,PryTorre,PryNumero,PryCreditoNo,PryCuentaDesembolso
0,1,ARPRO ARQUITECTOS INGENIEROS S.A.S,137,PRUEBAS REIMPLEMENTACION,NaN,NaN,1.0,Obras a Todo Costo,0,En Ejecucion,3,Caminos de Sie - Manzana 2,0,BOGOTÁ D.C.,NaN,NaN,NaN,NaN,Mar 6 2017 12:00AM,NaN,NaN,Entidad Propia,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,ARPRO ARQUITECTOS INGENIEROS S.A.S,203,Proyecto Ejemplo - Ventas,NaN,NaN,1.0,Obras a Todo Costo,2,Presupuesto,7,91 Octava,1,BOGOTÁ D.C.,1.0,6.0,1.0,NaN,Dic 30 2022 12:00AM,Davivienda,860002527.0,Entidad Propia,NaN,No,CR 8 90 87,NaN,NaN,NaN,NaN,NaN,2022-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,ARPRO ARQUITECTOS INGENIEROS S.A.S,207,Sangregado,2211401.0,Ed.Sangregado Directos,10.0,CBR - CLASE ARPRO ARQUITECTOS,2,Presupuesto,7,91 Octava,0,BOGOTÁ D.C.,1.0,6.0,1.0,NaN,Dic 31 2022 12:00AM,Davivienda,860002527.0,NaN,NaN,No,CR 8 91 07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-06-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,ARPRO ARQUITECTOS INGENIEROS S.A.S,218,VALVERDE OLIVO TORRE 1,NaN,NaN,9.0,CBR - CLASE FIDUCIARIA,2,Presupuesto,9,VALVERDE OLIVO,1,CHIA,NaN,NaN,NaN,NaN,Dic 31 2023 12:00AM,Davivienda,860002527.0,Accion Fiduciaria,NaN,No,CL 19 9 38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2020-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,ARPRO ARQUITECTOS INGENIEROS S.A.S,219,VALVERDE OLIVO TORRE 2,NaN,NaN,9.0,CBR - CLASE FIDUCIARIA,2,Presupuesto,9,VALVERDE OLIVO,1,CHIA,NaN,0.0,NaN,Mar 31 2023 12:00AM,Ene 1 1900 12:00AM,Davivienda,860002527.0,Accion Fiduciaria,NaN,No,CL 19 9 38,NaN,2023-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1900-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# diccionario de varibles

In [29]:
df_dic = columnas_y_llaves(esquemas=["ADP_DTM_DIM", "ADP_DTM_FACT"])


In [ ]:
# # Todo
# df_dic = columnas_pk_fk()

# print(df_dic["tipo_llave"].value_counts(dropna=False))

tipo_llave
None    6238
PK        14
Name: count, dtype: int64


In [30]:
df_dic.columns

Index(['esquema', 'nombre_tabla', 'nombre_columna', 'tipo_llave',
       'relacion_aTabla'],
      dtype='object')

In [31]:
len(df_dic['nombre_tabla'].unique())

50

In [ ]:
# df_dic.to_csv(r"C:\Users\Administrador\Desktop\reto\export\20251003\dicVars.csv", index=False)

In [21]:
df_dic['tipo_llave'].value_counts()

tipo_llave
NINGUNA    6238
PK           14
Name: count, dtype: int64

In [33]:
def listar_foreign_keys(engine):
    sql = text("""
    SELECT 
        fk.name                     AS nombre_fk,
        schp.name + '.' + tp.name   AS tabla_hija,
        cp.name                     AS columna_hija,
        schr.name + '.' + tr.name   AS tabla_padre,
        cr.name                     AS columna_padre
    FROM sys.foreign_keys fk
    JOIN sys.foreign_key_columns fkc
        ON fkc.constraint_object_id = fk.object_id
    JOIN sys.tables tp
        ON tp.object_id = fk.parent_object_id
    JOIN sys.schemas schp
        ON schp.schema_id = tp.schema_id
    JOIN sys.columns cp
        ON cp.object_id = tp.object_id
       AND cp.column_id = fkc.parent_column_id
    JOIN sys.tables tr
        ON tr.object_id = fk.referenced_object_id
    JOIN sys.schemas schr
        ON schr.schema_id = tr.schema_id
    JOIN sys.columns cr
        ON cr.object_id = tr.object_id
       AND cr.column_id = fkc.referenced_column_id
    ORDER BY tabla_hija, columna_hija;
    """)

    with engine.connect() as conn:
        df = pd.read_sql(sql, conn)

    return df

In [35]:
listar_foreign_keys(engine)

,nombre_fk,tabla_hija,columna_hija,tabla_padre,columna_padre


# Revisión Tablas exportadas

In [33]:
len(os.listdir('export/20251003'))

51

In [24]:
empresa = pd.read_csv(r"C:\Users\Administrador\Desktop\reto\export\20251003\ADP_DTM_DIM.Empresa.csv")
programacion = pd.read_csv(r"C:\Users\Administrador\Desktop\reto\export\20251003\ADP_DTM_FACT.Programacion.csv")

In [25]:
empresa.columns

Index(['SkIdEmpresa', 'NombreEmpresa', 'Nit', 'Direccion', 'Ref_IdEmpresa',
       'Ref_BdConfServidor'],
      dtype='object')

In [26]:
df_merge = pd.merge(empresa, programacion, on="SkIdEmpresa", how="inner")


In [28]:
df_merge.head()

,SkIdEmpresa,NombreEmpresa,Nit,Direccion,Ref_IdEmpresa,Ref_BdConfServidor,SkIdProyecto,SkIdActividad,SkIdFechaInicial,SkIdFechaFinal,Duracion,PorcentajeAsignado
0,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1,100269,1001000062763286,20250113,20250217,30,1.0
1,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1,100269,1001021208127655,20260727,20260810,10,0.0
2,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1,100269,1001070357588252,20261112,20270104,34,0.0
3,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1,100269,1001083471036420,20261130,20261204,4,0.0
4,100,ARPRO ARQUITECTOS INGENIEROS S.A.S,860067697,CRA 19 No 90-10,1,1,100269,1001124452537653,20260909,20260910,1,0.0


# Inconsistencias 

1. Cantidad Item `Tabla Proyeccion` revisar motivo de `NA` (campo vacio), solucion dianix = se eliminó la columna. **Utilidad: DB**
2. Revisar variable  `macroProyectoDescripcion` tabla `proyecto` campos vacios, solucion dianix = no pertenece a macro proyecto **Hipotesis:** Se asumio dado esata falta de información que este proyecto no pertenece a un macro proyecto.
3. Revisar variable `Fecha Modificacion` tabla `Insumo` campos vacios. Solucion dianix (cuestionable) = se buscó la fecha más antigua de todo el conjunto de datos (`Tabla_final`) y se reemplazó por una fecha más antigua que esta misma (**algo de diciembre del 2009**). ¿Que es lo que se modifica? El profe recomendó esta variable para traerla a valor presente.
4. Revisar variable `Insumo Descripcion` de la tabla `insumo` espacios vacios. Solucion Dianix: En los espacios vacios se imputo `No se encuentra una descripción disponible`. **Nombre del insumo**
5. Revisar `Agrupacion Descripcion` tabla `insumo` espacios vacios Solucion dianix: Se eliminó variable.
6. Revisar `Requiere Equipo` tabla `insumo` espacios vacios Solucion Dianix:  Se eliminó variable.
7. Revisar `Fecha Cracion` tabla `insumo` espacios vacios Solucion dianix (cuestionable) = se buscó la fecha más antigua de todo el conjunto de datos (`Tabla_final`) y se reemplazó por una fecha más antigua que esta misma (**algo de diciembre del 2009**).
8. Revisar `Valor Unitario` tabla `proyeccion` verificar si hay valores negativos. No hay solución Dianix ;(.
9. Revisar `Valor Total` tabla `proyeccion` verificar si hay valores negativos. No hay solución Dianix ;(.
10. El formato de fecha de las columnas de fecha anteriormente mencionadas se tuvo que modificar para que BQ lograra interpretarlas. 
11. Revisar si hay fechas relacionadas con los items. Se buscó en las tablas (`Proyeccion`, `Proyecto`, `Items`, `Insumo`, `capituloPresupuesto`)
12. Revisar consistencias en las fechas de los proyectos, es decir, revisar que  `Fecha Inicio` < `Fecha Elaboracion` < `Fecha Finalizacion`, todas estas fechas estan en la tabla `proyecto`, esto mismo rectificarlo para fechas de insumo anteriormente mencionado.

## 8

# Esquemas

In [ ]:
# df = pd.read_sql("SELECT * FROM ADP_DTM_FACT.Proyeccion", engine)
def describe_table(engine, schema, table):
    sql = text("""
    SELECT 
        c.TABLE_SCHEMA      AS schema_name,
        c.TABLE_NAME        AS table_name,
        c.COLUMN_NAME,
        c.DATA_TYPE,
        c.CHARACTER_MAXIMUM_LENGTH,
        c.IS_NULLABLE,
        c.COLUMN_DEFAULT
    FROM INFORMATION_SCHEMA.COLUMNS AS c
    WHERE c.TABLE_SCHEMA = :schema
      AND c.TABLE_NAME = :table
    ORDER BY c.ORDINAL_POSITION;
    """)
    with engine.connect() as conn:
        df = pd.read_sql(sql, conn, params={"schema": schema, "table": table})
    return df

# Ejemplo de uso


,schema_name,table_name,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,IS_NULLABLE,COLUMN_DEFAULT
0,ADP_DTM_FACT,Proyeccion,SkIdEmpresa,smallint,NaN,YES,None
1,ADP_DTM_FACT,Proyeccion,SkIdProyecto,int,NaN,YES,None
2,ADP_DTM_FACT,Proyeccion,SkIdCapitulo,bigint,NaN,YES,None
3,ADP_DTM_FACT,Proyeccion,SkIdItems,bigint,NaN,YES,None
4,ADP_DTM_FACT,Proyeccion,SkIdInsumo,bigint,NaN,YES,None
5,ADP_DTM_FACT,Proyeccion,SkIdReforma,int,NaN,YES,None
6,ADP_DTM_FACT,Proyeccion,SkIdUsuario,int,NaN,YES,None
7,ADP_DTM_FACT,Proyeccion,SkIdFecha,int,NaN,YES,None
8,ADP_DTM_FACT,Proyeccion,SkIdFecha Real,int,NaN,YES,None
9,ADP_DTM_FACT,Proyeccion,SkIdEstado,int,NaN,YES,None


In [37]:
df_desc = describe_table(engine, "ADP_DTM_FACT", "Proyeccion")
df_desc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19 entries, 0 to 18
Data columns (total 7 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   schema_name               19 non-null     object 
 1   table_name                19 non-null     object 
 2   COLUMN_NAME               19 non-null     object 
 3   DATA_TYPE                 19 non-null     object 
 4   CHARACTER_MAXIMUM_LENGTH  3 non-null      float64
 5   IS_NULLABLE               19 non-null     object 
 6   COLUMN_DEFAULT            0 non-null      object 
dtypes: float64(1), object(6)
memory usage: 1.2+ KB


In [35]:
df_desc = describe_table(engine, "ADP_DTM_DIM", "Empresas")
df_desc

,schema_name,table_name,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,IS_NULLABLE,COLUMN_DEFAULT


In [60]:
import numpy as np

def tables_describe(schemas:list):
    tablas = listar_tablas()
    tablasUsar = tablas[tablas['TABLE_SCHEMA'].isin( ['ADP_DTM_DIM','ADP_DTM_FACT'])]
    tables_description = {}
    for schema in schemas:
        tablas = list(tablasUsar[tablasUsar['TABLE_SCHEMA'] == schema]['TABLE_NAME'])
        for tabla in tablas:
            df_description = describe_table(engine, schema, tabla)
            if df_description.shape[0] == 0:
                nueva_fila = {
                    "schema_name": schema,
                    "table_name": tabla
                }
                df_fila = pd.DataFrame([nueva_fila]).reindex(columns=df_desc.columns)
                df_fila = df_fila.fillna("NA")
                if "CHARACTER_MAXIMUM_LENGTH" in df_fila.columns:
                    df_fila["CHARACTER_MAXIMUM_LENGTH"] = np.nan
                df_description = pd.concat([df_desc, df_fila], ignore_index=True)

            tables_description[tabla] = df_description

    df_final = pd.concat(tables_description.values(), ignore_index=True)
    return df_final

In [44]:
schemas = list(tablasUsar['TABLE_SCHEMA'].unique())
schemas

['ADP_DTM_DIM', 'ADP_DTM_FACT']

In [61]:
schemas = list(tablasUsar['TABLE_SCHEMA'].unique())
df_descriptions = tables_describe(schemas)

C:\Users\Administrador\AppData\Local\Temp\ipykernel_5100\1671115763.py:24: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_final = pd.concat(tables_description.values(), ignore_index=True)


In [63]:
len(df_descriptions['table_name'].unique())

50

In [65]:
df_descriptions[df_descriptions['table_name'] == 'Empresa']

,schema_name,table_name,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,IS_NULLABLE,COLUMN_DEFAULT
21,ADP_DTM_DIM,Empresa,SkIdEmpresa,smallint,NaN,NO,None
22,ADP_DTM_DIM,Empresa,NombreEmpresa,varchar,140.0,YES,None
23,ADP_DTM_DIM,Empresa,Nit,varchar,40.0,YES,None
24,ADP_DTM_DIM,Empresa,Direccion,varchar,200.0,YES,None
25,ADP_DTM_DIM,Empresa,Ref_IdEmpresa,smallint,NaN,YES,None
26,ADP_DTM_DIM,Empresa,Ref_BdConfServidor,smallint,NaN,YES,None


In [67]:
df_descriptions.shape

(531, 7)

## Exportar esquemas

In [66]:
df_descriptions.to_csv(
    "tableDescriptions.csv",
    index=False,
    encoding="utf-8-sig"
)
